In [49]:
import plotly.express as px
import matplotlib as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast


biwenger = pd.read_csv("../data/biwenger.csv", index_col=[0])
biwenger["value"] = biwenger.value.str.replace(',', '').astype(int)
lineups = pd.read_csv("../data/lineups.csv", sep=";", index_col=[0])
positions = ["Goalkeeper", "Defender", "Midfielder", "Forward"]

In [83]:
th_props = [
  ('font-size', '11px'),
  ('text-align', 'center'),
  ('font-weight', 'bold'),
  ('color', '#6d6d6d'),
  ('background-color', '#f7f7f9')
]

td_props = [
  ('font-size', '11px')
]

styles = [
  dict(selector="th", props=th_props),
  dict(selector="td", props=td_props)
]

green_cm = sns.light_palette("#44ab42", as_cmap=True)

def color_text_distribution(value, mean, std, positive=True):
    if value <= mean-(1.5*std):
        color = 'red' if positive else 'green'
    elif value >= mean+(1.5*std):
        color = 'green' if positive else 'red'
    else:
        color = 'black'
    return 'color: %s' % color

def top_players_table(players, position, k=15):
    selected_columns = ["name", "games", "points", "value", "cards", "points_per_million", "points_per_game", "pct_lineup"]
    top = players.loc[players.position == position, selected_columns].sort_values("pct_lineup", ascending=False).head(k)

    card_stats = players.loc[players.position == position, "cards"].describe()
    ppg_stats = players.loc[players.position == position, "points_per_game"].describe()
    ppm_stats = players.loc[players.position == position, "points_per_million"].describe()

    return (top.style
               .applymap(lambda x: "font-weight: bold", subset=['name'])
               .background_gradient(cmap=green_cm, subset=['pct_lineup'])
               .applymap(lambda x: color_text_distribution(x, card_stats["mean"], card_stats["std"], positive=False), subset=['cards'])
               .applymap(lambda x: color_text_distribution(x, ppg_stats["mean"], ppg_stats["std"]), subset=['points_per_game'])
               .applymap(lambda x: color_text_distribution(x, ppm_stats["mean"], ppm_stats["std"]), subset=['points_per_million'])
               .set_caption('Top {0}s:'.format(position))
               .format({'points_per_game': "{:.3}", 'points_per_million': "{:.4}"})
               .set_table_styles(styles))

def bubble_chart(players, position):
    top = players.loc[players.position == position]
    fig = px.scatter(top, x="points_per_game", y="points_per_million", size="pct_lineup", color="value", hover_name="name")
    fig.update_xaxes(title_text='Points per Game')
    fig.update_yaxes(title_text='Points per Million')
    fig.show()

# Position Analysis

In [84]:
position_stats = []
for position in positions:
    # Points average and sum
    str_points = lineups["{0}_points".format(position.lower())]
    point_sum = str_points.apply(ast.literal_eval).apply(np.sum).mean()
    point_avg = str_points.apply(ast.literal_eval).apply(np.mean).mean()
    
    # Value average and sum
    str_value = lineups["{0}_values".format(position.lower())]
    value_sum = str_value.apply(ast.literal_eval).apply(np.sum).mean()
    value_avg = str_value.apply(ast.literal_eval).apply(np.mean).mean()
    value_pct = value_sum/lineups["total_value"]
    
    position_stats.append({
        "position": position,
        "sum_points": point_sum,
        "avg_points": point_avg,
        "sum_value": value_sum,
        "avg_value": value_avg,
        "pct_value": value_pct.mean()
    })
    
position_stats = pd.DataFrame(position_stats)
position_stats

,avg_points,avg_value,pct_value,position,sum_points,sum_value
0,155.790,3.425900e+06,0.061241,Goalkeeper,155.79,3425900.0
1,133.935,2.334600e+06,0.166933,Defender,535.74,9338400.0
2,148.975,3.243717e+06,0.347907,Midfielder,893.85,19462300.0
3,222.840,7.979200e+06,0.427907,Forward,668.52,23937600.0


# Player Analysis

In [85]:
player_appearances = []
for _, row in lineups.iterrows():
    for position in positions:
        str_players = row["{0}s".format(position.lower())]
        for player in ast.literal_eval(str_players):
            player_appearances.append({"name": player})

player_counts = pd.DataFrame(player_appearances).groupby("name").size().reset_index(name='lineups')
players = pd.merge(player_counts, biwenger, how='left', on='name')

players["points_per_million"] = players["points"] / players["value"] * 1000000
players["pct_lineup"] = players["lineups"] / lineups.shape[0] * 100
players["points_per_game"] = players["points"] / players["games"]

## Goalkeepers

In [86]:
top_players_table(players, "Goalkeeper")

,name,games,points,value,cards,points_per_million,points_per_game,pct_lineup
56,David Soria,37,178,3580000,0,49.72,4.81,13
161,Pacheco,35,173,3610000,3,47.92,4.94,12
50,Cuéllar,34,148,2200000,5,67.27,4.35,11
158,Oblak,37,196,6480000,0,30.25,5.3,11
203,Ter Stegen,35,167,4720000,0,35.38,4.77,11
62,Diego López,38,160,2980000,1,53.69,4.21,9
11,Asenjo,32,157,2560000,0,61.33,4.91,8
139,Masip,35,150,2810000,1,53.38,4.29,6
93,Herrerín,31,94,1290000,1,72.87,3.03,5
65,Dmitrovic,24,106,2220000,3,47.75,4.42,4


In [87]:
bubble_chart(players, "Goalkeeper")

## Defenders

In [88]:
top_players_table(players, "Defender")

,name,games,points,value,cards,points_per_million,points_per_game,pct_lineup
185,Rubén Peña,31,174,2440000,4,71.31,5.61,14
20,Bernardo,31,124,1850000,6,67.03,4.0,12
132,Mandi,35,158,2340000,10,67.52,4.51,12
109,Jonathan Silva,30,150,1890000,9,79.37,5.0,11
4,Antunes,28,126,370000,5,340.5,4.5,10
91,Hermoso,32,178,5620000,7,31.67,5.56,10
122,Laguardia,36,170,3330000,12,51.05,4.72,10
53,Damián Suárez,36,152,2550000,12,59.61,4.22,10
35,Carriço,24,125,1190000,10,105.0,5.21,10
111,Jordi Alba,36,200,6880000,6,29.07,5.56,9


In [89]:
bubble_chart(players, "Defender")

## Midfielders

In [90]:
top_players_table(players, "Midfielder")

,name,games,points,value,cards,points_per_million,points_per_game,pct_lineup
162,Parejo,36,255,11420000,6,22.33,7.08,22
1,Alcaraz,34,176,3210000,13,54.83,5.18,19
110,Jordan,36,201,4620000,10,43.51,5.58,19
32,Canales,32,214,6630000,7,32.28,6.69,18
172,Rakitic,34,195,5530000,7,35.26,5.74,16
108,Joaquín,30,175,2590000,5,67.57,5.83,16
213,Vidal,33,174,3360000,7,51.79,5.27,16
38,Cazorla,35,215,6860000,2,31.34,6.14,15
31,Campaña,36,194,4200000,12,46.19,5.39,15
134,Marc Roca,35,173,3570000,9,48.46,4.94,14


In [91]:
bubble_chart(players, "Midfielder")

## Forwards

In [92]:
top_players_table(players, "Forward")

,name,games,points,value,cards,points_per_million,points_per_game,pct_lineup
145,Messi,34,428,22790000,3,18.78,12.6,45
112,Jorge Molina,38,244,6560000,2,37.2,6.42,22
103,Jaime Mata,34,209,6440000,10,32.45,6.15,16
40,Charles,34,185,2500000,4,74.0,5.44,14
129,Luis Suárez,33,262,11740000,5,22.32,7.94,13
19,Benzema,36,263,10710000,1,24.56,7.31,13
59,Dembélé,29,179,3990000,1,44.86,6.17,12
68,En-Nesyri,31,161,3100000,7,51.94,5.19,10
22,Borja Iglesias,37,239,9660000,5,24.74,6.46,9
18,Ben Yedder,35,246,8320000,4,29.57,7.03,9


In [93]:
bubble_chart(players, "Forward")

## OPTA Analysis